In [1]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk import word_tokenize, pos_tag
from wordcloud import WordCloud
from collections import Counter

In [2]:
# Nạp dữ liệu
df = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None)
# df = pd.read_csv('D:/PROJECT/Classification_Status/Data/training.1600000.processed.noemoticon.csv', encoding='latin-1', header=None) #thay đổi đường dẫn tới file dữ liệuliệuc
# Đặt tên cho các cột
df.columns = ['target', 'id', 'date', 'query', 'user', 'text']

# Xem trước dữ liệu
df.head()

# Giữ lại 2 cột quan trọng là 'polarity' và 'text'
df = df[['target', 'text']]



In [ ]:
# Loại bỏ các ký tự đặc biệt, số và chuyển thành chữ thường
def clean_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Loại bỏ URL
    text = re.sub(r'\@\w+|\#', '', text)  # Loại bỏ @ và #
    text = re.sub(r'[^A-Za-z\s]', '', text)  # Chỉ giữ lại chữ cái
    text = text.lower()  # Chuyển thành chữ thường
    return text

df['clean_text'] = df['text'].apply(clean_text)

# Loại bỏ các stop words
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    if isinstance(text, str):  # Kiểm tra nếu text là chuỗi
        tokens = word_tokenize(text)
        filtered_tokens = [word for word in tokens if word not in stop_words]
        return ' '.join(filtered_tokens)
    else:
        return ''  

df['clean_text'] = df['clean_text'].apply(remove_stopwords)

# Xem dữ liệu sau khi làm sạch
df.head()

In [ ]:
# Tách các câu tích cực và tiêu cực
negative_texts = df['clean_text'][:800000].dropna().astype(str)  
positive_texts = df['clean_text'][800000:].dropna().astype(str)  

# Đếm tần suất từ vựng
positive_words = ' '.join(positive_texts).split()
negative_words = ' '.join(negative_texts).split()

# Đếm số lượng từ phổ biến nhất
positive_word_freq = Counter(positive_words)
negative_word_freq = Counter(negative_words)

# 10 từ phổ biến nhất trong câu tích cực
print("Top 10 từ phổ biến trong các câu tích cực:")
print(positive_word_freq.most_common(10))

# 10 từ phổ biến nhất trong câu tiêu cực
print("Top 10 từ phổ biến trong các câu tiêu cực:")
print(negative_word_freq.most_common(10))


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

# Hàm để lấy POS tags
def pos_tagging(texts):
    all_tags = []
    for text in texts:
        tokens = word_tokenize(text)
        tags = pos_tag(tokens)
        all_tags.extend([tag for word, tag in tags])
    return all_tags

# POS tagging cho câu tích cực và tiêu cực
positive_pos_tags = pos_tagging(positive_texts)
negative_pos_tags = pos_tagging(negative_texts)

# Đếm tần suất POS tags
positive_pos_freq = Counter(positive_pos_tags)
negative_pos_freq = Counter(negative_pos_tags)

# Xem các POS tags phổ biến
print("Top POS tags trong câu tích cực:")
print(positive_pos_freq.most_common(10))

print("Top POS tags trong câu tiêu cực:")
print(negative_pos_freq.most_common(10))


In [ ]:
# Trực quan hóa từ vựng phổ biến bằng seaborn
def plot_word_frequency(word_freq, title):
    # Lấy 10 từ phổ biến nhất
    common_words = word_freq.most_common(10)
    if not common_words:  # Kiểm tra nếu không có từ nào
        print("Không có từ nào để hiển thị.")
        return

    words = [word for word, freq in common_words]
    freqs = [freq for word, freq in common_words]

    # Vẽ biểu đồ
    plt.figure(figsize=(10, 6))
    sns.barplot(x=freqs, y=words)
    plt.title(title)
    plt.xlabel('Tần suất')
    plt.ylabel('Từ vựng')
    plt.show()
    
print("Số từ tích cực:", len(positive_word_freq))
print("Số từ tiêu cực:", len(negative_word_freq))
# Biểu đồ cho từ vựng tích cực
plot_word_frequency(positive_word_freq, 'Từ vựng phổ biến trong câu tích cực')

# Biểu đồ cho từ vựng tiêu cực
plot_word_frequency(negative_word_freq, 'Từ vựng phổ biến trong câu tiêu cực')


In [ ]:
# Tạo WordCloud
def create_wordcloud(word_freq, title):
    if not word_freq:  # Kiểm tra nếu không có từ nào
        print("Không có từ nào để tạo WordCloud.")
        return

    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')  # Tắt trục
    plt.title(title)
    plt.show()

# WordCloud cho từ vựng tích cực
create_wordcloud(positive_word_freq, 'WordCloud cho các từ tích cực')

# WordCloud cho từ vựng tiêu cực
create_wordcloud(negative_word_freq, 'WordCloud cho các từ tiêu cực')


In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import BartTokenizer, BartForConditionalGeneration

In [ ]:
# Load tokenizer và model BART
bart_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
bart_model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [ ]:
# Kiểm tra xem có sử dụng GPU không
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bart_model.to(device)


In [ ]:
def summarize_text(text, summary_length_ratio=0.3, min_length=20, max_length=150):
    word_count = len(text.split())

    if word_count < 10:
        return text

    inputs = bart_tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=1024).to(device)
    original_word_count = word_count
    dynamic_max_length = min(max_length, max(1, int(original_word_count * summary_length_ratio)))
    
    if dynamic_max_length < min_length:
        dynamic_max_length = min_length

    summary_ids = bart_model.generate(
        inputs['input_ids'],
        max_length=dynamic_max_length,
        min_length=min_length,
        length_penalty=2.0,
        num_beams=2,
        early_stopping=True
    )
    
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    return summary

In [ ]:
# Chuyển đổi nhãn 4 thành 1
df['target'] = df['target'].replace(4, 1)
# Kiểm tra lại nhãn trong toàn bộ DataFrame
print("Unique labels in dataset:", set(df['target']))

# Đếm số lượng dữ liệu tích cực (nhãn 1) và tiêu cực (nhãn 0)
label_counts = df['target'].value_counts()

print("Số lượng dữ liệu tích cực và tiêu cực:")
print(label_counts)

# Tách số liệu cụ thể
positive_count = label_counts[1]  # Dữ liệu tích cực (nhãn 1)
negative_count = label_counts[0]  # Dữ liệu tiêu cực (nhãn 0)

print(f"Số lượng dữ liệu tích cực: {positive_count}")
print(f"Số lượng dữ liệu tiêu cực: {negative_count}")




In [ ]:
# Lọc dữ liệu tích cực (target == 1) và tiêu cực (target == 0)
positive_texts = df[df['target'] == 1]  # Dữ liệu tích cực
negative_texts = df[df['target'] == 0]  # Dữ liệu tiêu cực

# Lấy 5000 dòng dữ liệu tích cực và tiêu cực ngẫu nhiên
positive_sample = positive_texts.sample(n=30000, random_state=42)  
negative_sample = negative_texts.sample(n=30000, random_state=42)  

# Kết hợp dữ liệu tích cực và tiêu cực đã lấy mẫu
sampled_df = pd.concat([positive_sample, negative_sample])

# Xem số lượng dữ liệu
print(f"Tổng số dữ liệu sau khi kết hợp: {len(sampled_df)}")

In [ ]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_texts, test_texts, train_labels, test_labels = train_test_split(
    sampled_df['clean_text'].tolist(),    
    sampled_df['target'].tolist(),        
    test_size=0.2,                       
    random_state=42
)

print(f"Số lượng dữ liệu huấn luyện: {len(train_texts)}")
print(f"Số lượng dữ liệu kiểm tra: {len(test_texts)}")


In [ ]:
print("Bắt đầu tóm tắt dữ liệu tập train...")

train_texts_summarized = [summarize_text(text) for text in train_texts]
print("Hoàn thành tóm tắt dữ liệu cho tập train.")
print(f"Đã có {len(train_texts_summarized)} dòng dữ liệu được tóm tắt từ tập train.")

print("Bắt đầu tóm tắt dữ liệu tập test...")
test_texts_summarized = [summarize_text(text) for text in test_texts]
print(f"Đã có {len(test_texts_summarized)} dòng dữ liệu được tóm tắt từ tập test.")

print("Hoàn thành tóm tắt dữ liệu.")

In [ ]:
# Tải tokenizer và mô hình BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Token hóa dữ liệu tóm tắt
train_encodings = tokenizer(train_texts_summarized, truncation=True, padding=True, max_length=250)
test_encodings = tokenizer(test_texts_summarized, truncation=True, padding=True, max_length=250)

In [ ]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataset = SentimentDataset(train_encodings, train_labels)
test_dataset = SentimentDataset(test_encodings, test_labels)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    acc = accuracy_score(p.label_ids, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [ ]:
!pip install pytorch-optimizer

In [ ]:
import torch
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback, AdamW, get_scheduler
from pytorch_optimizer import RAdam

print(torch.cuda.is_available())  
print(torch.cuda.device_count())  

# Định nghĩa các tham số cho huấn luyện
training_args = TrainingArguments(
    output_dir='./results',                
    report_to="none",                     
    num_train_epochs=8,                   
    per_device_train_batch_size=32,        
    per_device_eval_batch_size=32,        
    warmup_steps=446,                     
    weight_decay=0.0001312884113835475,                    
    logging_dir='./logs',                 
    learning_rate=4.628809598490191e-05,                   
    adam_beta1=0.8697519534926305,                       
    adam_beta2=0.9954373236804777,                     
    adam_epsilon=3.1399388605341155e-08,                    
    logging_steps=10,                     
    evaluation_strategy="steps",          
    save_steps=500,                       
    eval_steps=500,                       
    load_best_model_at_end=True,  
    metric_for_best_model="accuracy",           
    greater_is_better=True, 
    save_total_limit=2                    
)

# Thêm EarlyStoppingCallback
callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]

# Tạo optimizer và scheduler cho mô hình
optimizer = RAdam([
    {'params': model.bert.encoder.layer[:6].parameters(), 'lr': 7.817983122737902e-06},
    {'params': model.bert.encoder.layer[6:9].parameters(), 'lr':  6.782664240283009e-06},
    {'params': model.bert.encoder.layer[9:].parameters(), 'lr': 3.1679835689436935e-05},
    {'params': model.classifier.parameters(), 'lr': 3.8891509085731684e-05}
], betas=(0.9, 0.999),weight_decay=0.0001312884113835475)

# Tính số bước huấn luyện
num_training_steps = len(train_dataset) // training_args.per_device_train_batch_size * training_args.num_train_epochs

# Tạo scheduler cho learning rate
lr_scheduler = get_scheduler(
    "cosine",
    optimizer=optimizer,
    num_warmup_steps=training_args.warmup_steps,
    num_training_steps=num_training_steps
)

# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,          
    eval_dataset=test_dataset,  
    compute_metrics=compute_metrics,
    callbacks=callbacks,                  
    optimizers=(optimizer, lr_scheduler)  
)


In [ ]:
# Bước 1: Huấn luyện mô hình
trainer.train()

# Bước 2: Dự đoán nhãn cho tập kiểm tra
predictions, labels, _ = trainer.predict(test_dataset)
predictions = np.argmax(predictions, axis=1)

# Bước 3: Tạo bảng so sánh giữa nhãn gốc và nhãn dự đoán
comparison_df = pd.DataFrame({
    'Original_Text': test_texts,           # Câu gốc từ tập kiểm tra
    'Original_Label': test_labels,         # Nhãn gốc từ cột target
    'Predicted_Label': predictions         # Nhãn dự đoán từ BERT
})

# In ra 20 dòng đầu tiên của bảng so sánh
print(comparison_df.head(20))

# Xuất ra file CSV để kiểm tra
comparison_df.to_csv('bert_comparison_results.csv', index=False)

# Bước 4: In báo cáo phân loại
print(classification_report(test_labels, predictions))

In [ ]:
import matplotlib.pyplot as plt

# Lấy các giá trị loss từ log_history
train_loss = []
eval_loss = []
steps = []  # Số bước log

# Duyệt qua log_history và lấy loss và eval_loss cùng với steps
for log in trainer.state.log_history:
    if 'loss' in log:
        train_loss.append(log['loss'])
        steps.append(log['step'])  # Lưu số bước tương ứng với loss
    if 'eval_loss' in log:
        eval_loss.append(log['eval_loss'])

# Vẽ biểu đồ Loss theo Steps (thay vì Epochs)
plt.figure(figsize=(10, 5))
plt.plot(steps, train_loss, label='Training Loss')
plt.plot(steps[:len(eval_loss)], eval_loss, label='Evaluation Loss')  # Cắt eval_loss nếu cần
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.title('Training & Evaluation Loss over Steps')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np# Bước 5: Tính toán confusion matrix
cm = confusion_matrix(test_labels, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# Tính toán độ chính xác
accuracy = accuracy_score(test_labels, predictions)
print(f"Accuracy: {accuracy:.4f}")

# Tính toán precision, recall, và f1-score
precision = precision_score(test_labels, predictions, average='weighted')
recall = recall_score(test_labels, predictions, average='weighted')
f1 = f1_score(test_labels, predictions, average='weighted')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Tính toán AUC-ROC
roc_auc = roc_auc_score(test_labels, predictions)
print(f"AUC-ROC: {roc_auc:.4f}")

# Vẽ biểu đồ ROC
fpr, tpr, _ = roc_curve(test_labels, predictions)
plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np# Bước 5: Tính toán confusion matrix
cm = confusion_matrix(test_labels, predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# Tính toán độ chính xác
accuracy = accuracy_score(test_labels, predictions)
print(f"Accuracy: {accuracy:.4f}")

# Tính toán precision, recall, và f1-score
precision = precision_score(test_labels, predictions, average='weighted')
recall = recall_score(test_labels, predictions, average='weighted')
f1 = f1_score(test_labels, predictions, average='weighted')

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Tính toán AUC-ROC
roc_auc = roc_auc_score(test_labels, predictions)
print(f"AUC-ROC: {roc_auc:.4f}")

# Vẽ biểu đồ ROC
fpr, tpr, _ = roc_curve(test_labels, predictions)
plt.figure()
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='red', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Save model and tokenizer
model.save_pretrained("/kaggle/working/bert_model")
tokenizer.save_pretrained("/kaggle/working/bert_model")


In [ ]:
import shutil

# Nén thư mục bert_model thành tệp ZIP
shutil.make_archive("/kaggle/working/bert_model", 'zip', "/kaggle/working/bert_model")

print("Mô hình đã được nén thành công thành tệp ZIP.")
